<a href="https://colab.research.google.com/github/Rishwantthi/SYNAPSE/blob/main/Imagetxtaudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies
!pip install transformers accelerate bitsandbytes gTTS gradio --quiet
!apt-get install -y tesseract-ocr --quiet
!pip install pytesseract opencv-python --quiet

from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import pytesseract
import torch
from gtts import gTTS
import gradio as gr
import cv2
import numpy as np

# Load BLIP model (single transformer: ViT + BERT)
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

# --- OCR Preprocessing ---
def preprocess_for_ocr(pil_image):
    img = np.array(pil_image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
    img = cv2.GaussianBlur(img, (5,5), 0)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img

def extract_ocr_text(pil_image):
    preprocessed_img = preprocess_for_ocr(pil_image)
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(preprocessed_img, config=custom_config)
    return text.strip()

# --- Main Pipeline ---
def process_image(image):
    # 1. Generate Caption
    inputs = processor(image, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    caption = processor.decode(out[0], skip_special_tokens=True)

    # 2. Extract OCR (for text display only)
    ocr_text = extract_ocr_text(image)

    # 3. Convert Caption to Speech (OCR NOT included)
    tts = gTTS(caption)
    audio_path = "caption_audio.mp3"
    tts.save(audio_path)

    return caption, ocr_text, audio_path

# --- Gradio UI ---
demo = gr.Interface(
    fn=process_image,
    inputs=gr.Image(type="pil"),
    outputs=[
        gr.Textbox(label="Image Caption (BLIP)"),
        gr.Textbox(label="OCR Extracted Text"),
        gr.Audio(label="Caption Audio Output")
    ],
    title="ImageSpeak++ (Hackathon Edition)",
    description="Upload an image → BLIP caption + OCR text (display only) + audio narration of caption."
)

demo.launch(debug=True)


Reading package lists...
Building dependency tree...
Reading state information...
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://249a9e38d42776815a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


`cache.key_cache[idx]` is deprecated and will be removed in v4.56.0. Use `cache.layers[idx].keys` instead.
`cache.value_cache[idx]` is deprecated and will be removed in v4.56.0. Use `cache.layers[idx].values` instead.
